In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN_OV.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:

for index, row in df.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)
    
    algo = "svc"
    
    test = str(algo) # change before run
    
    level = "word"
    
    iterations = 30

    dropout = 0.5

    samples = 3200

    dimensions = 200

    loc = authorList.index(author_id)

    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

    print("Current test: %s" % (str(printstate)))
    
    import UpdateDB as db
    case = db.checkOldCNNDiffBoth(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                  samples = samples, iterations = iterations, dropout = dropout,
                                  test = test)
    
    if case == False:
        
        print("Running: %12s" % (str(printstate)))
        
        import StyloNeural as Stylo
        (labels_index, history, train_acc_cnn, val_acc_cnn, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], 
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        (predYList_cnn, predY_cnn, testY_cnn) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
            
        del Stylo
        
        from keras import backend as K
        K.clear_session()

        from IPython.display import clear_output

        clear_output()

        import StyloNeuralML as Stylo
        (labels_index, train_acc_ml, val_acc_ml, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], algo = algo,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, dropout = dropout)

        (predYList_ml, predY_ml, testY_ml, predEntro) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index, algo = algo,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            dropout = dropout, predYList_cnn = predYList_cnn)

        del Stylo

        from keras import backend as K
        K.clear_session()

        loc = testY_cnn
        
        test_acc_cnn = 0.0
        
        test_acc_cnn = predY_cnn[loc]
        
        test_bin_cnn = 0
        
        if(predY_cnn.tolist().index(max(predY_cnn)) == testY_cnn):
            test_bin_cnn = 1
        
        loc = testY_ml
        
        test_acc_ml = 0.0
        
        test_acc_ml = predY_ml[loc]
        
        test_bin_ml = 0
        
        if(predY_ml.tolist().index(max(predY_ml)) == testY_ml):
            test_bin_ml = 1
        
        # loc = testY_ml
        
        # test_acc = 0.0
        
        # test_acc = predEntro[loc]
        
        # test_bin = 0
        
        if(predEntro.tolist().index(max(predEntro)) == testY_cnn):
            test_bin = 1
            
        import UpdateDB as db
        case = db.updateresultOldCNNDiffBoth(
            doc_id = doc_id, candidate = candidate, dimensions = dimensions,
            samples = samples, iterations = iterations, dropout = dropout, 
            train_acc_cnn = train_acc_cnn, val_acc_cnn = val_acc_cnn,
            test_acc_cnn = test_acc_cnn, test_bin_cnn = test_bin_cnn, 
            train_acc_ml = train_acc_ml, val_acc_ml = val_acc_ml,
            test_acc_ml = test_acc_ml, test_bin_ml = test_bin_ml, 
            test = test)

        import time
        time.sleep(10)

        from IPython.display import clear_output

        clear_output()
        
    else:
        print("Skipped: %12s" % (str(printstate)))

# import pandas as pd
# df = pd.DataFrame(output)
# df.to_csv("styloout.csv", index = False, encoding='utf-8')

import time
time.sleep(10)

Current test: doc_id = 769, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = svc
Execution completed
Running: doc_id = 769, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = svc
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 117
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (12921, 2)
Author:    21  Size:  7467
Author:    59  Size:  3939
Author:   111  Size:  1515
Min: 1515
Max: 7467
Authors [21, 59, 111].
Found 4545 texts.
Found 4545 labels.
Found 65597 unique tokens.
Shape of data tensor: (4545, 1000)
Shape of label tensor: (4545, 3)
Done compiling.
Train on 3636 samples, validate on 909 samples
Epoch 1/30
3636/3636 [==============================] - 39s - loss: 0.8885 - acc: 0.5583 - val_loss: 0.8053 - val_acc: 0.5996
Epoch 2/30
3636/3636 [==============================] - 39s - loss: 0.4945 - a

In [ ]:
%tb